<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/R/colab/Cytoscape-ecosystem-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cytoscape Ecosystem Tutorial
## Alexander Pico


*The R markdown is available. [download the Rmd from GitHub](https://raw.githubusercontent.com/cytoscape/cytoscape-automation/master/for-scripters/R/notebooks/ismb-network-analysis-tutorial.Rmd).*

# Cytoscape Ecosytem
Cytoscape is a well-known bioinformatics tool for displaying and exploring biological networks. The Cytoscape Ecosytem extends beyond the desktop software to include web apps (like cytoscape.js), community-contributed collections of networks (NDEx) and apps (AppStore), and the CyREST programmatic interface. Programmatic access and interactive display via R, Python and JS enable a braod range of applications in network anlaysis and visualization, leveraging the Cytoscape Ecosystem.

# Installation
**RCy3** is a Bioconductor package that connects R to a locally running instance of the Cytoscape desktop software via CyREST.

In [ ]:
if(!"RCy3" %in% installed.packages()){
    install.packages("BiocManager")
    BiocManager::install("RCy3")
}
library(RCy3)

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)

In [ ]:
# Connect to local Cytoscape
browserClientJs <- getBrowserClientJs()
IRdisplay::display_javascript(browserClientJs)

The whole point of RCy3 is to connect with Cytoscape. You will need to install and launch Cytoscape:

* Download the latest Cytoscape from http://www.cytoscape.org/download.php
* Complete installation wizard
* Launch Cytoscape

Confirm your connection with the *cytoscapePing* funciton:

In [ ]:
cytoscapePing()

# PART I
Here, we will access NDEx and control Cytoscape using just a few of the hundreds of methods available in RCy3.

## Load Network from NDEx
We will import a network from NDEx into the Cytoscape environment. Networks are provided a Universal Unique ID (UUID) by NDEx that can be used to reliably reference a particular network. The import method returns a Cytoscape Session Unique ID (SUID) for the imported network, enabling us to reference it in subsequent steps within a given Cytoscape session.

In [ ]:
NETWORK_UUID = 'b1e9a489-bbe7-11ea-aaef-0ac135e8bacf' # UUID provided by NDEx
network_suid <- importNetworkFromNDEx(NETWORK_UUID)

## Copy Visual Style
One of the easiest ways to apply a visual style to a network is to copy that style from another network. This lets you work on the style of your network in Cytoscape using the interactive style editor, then save the network to NDEx for future use as a template.

The name of the template network becomes the name of its style and the *set* method assigns that style to another network, specified by that network's SUID.

In [ ]:
STYLE_NETWORK_UUID = 'b1c1aa27-bbe7-11ea-aaef-0ac135e8bacf'
style_network_suid <- importNetworkFromNDEx(STYLE_NETWORK_UUID)
style_network_name <- getNetworkName(style_network_suid)
setVisualStyle(paste0(style_network_name,'-Style'), network=network_suid)
setCurrentNetwork(network_suid)

## Apply Layout
We can easily access the layouts available in Cytoscape.

In [ ]:
getLayoutNames() # explore available layouts
layoutNetwork('force-directed', network=network_suid)

Cytoscape layouts also have parameters that can be tuned for each network

In [ ]:
getLayoutPropertyNames('force-directed') # explore parameter options
layoutNetwork('force-directed defaultSpringCoefficient=1E-5', network=network_suid)

## Save Network to NDEx
There is a one-step method to save networks from Cytoscape to NDEx. You can also control whether the network is publicly accessible or private to your account.

**Note:** By itself, "public" does not mean that users can find the network by searching NDEx. This is so that searches are not cluttered by networks that were convenient to make public but which are not intended for general use. To make a network findable in searches, you need to go to your account on the NDEx site and change that setting. In the context of a tutorial where you access NDEx programmatically, you can see how important this is: no one wants to find the 20,000 networks you accidentally loaded to your account because of a bug in your code.

In [ ]:
new_name <- paste0(getNetworkName(network_suid),'-tutorial')
renameNetwork(new_name)

In [ ]:
USERNAME <- readline('Enter your NDEx username: ')
PASSWORD <- readline('Enter your NDEx password: ')
new_network_uuid <- exportNetworkToNDEx(USERNAME, PASSWORD, isPublic=FALSE, network=network_suid)

# PART II
In addition to working with networks from NDEx, you can also load networks from local or hosted flatfiles in a variety of formats and annotate them with data, just like you would using the Cytoscape GUI.

## Load PPI Network
Tabular data can be read in as a dataframe and then loaded as a network in Cytoscape.

**Note:** A column named "source" automatically becomes the source node column; "target" becomes the target node column, and "interaction" becomes the edge interactions column. All other columns become edge attributes.

In [ ]:
PPI_DATA_URL = 'https://raw.githubusercontent.com/cytoscape/cytoscape-automation/master/for-scripters/Python/data/ap-ms-demo-data.csv'
ppi_data <- read.csv(PPI_DATA_URL, stringsAsFactors = F)
ppi_data
colnames(ppi_data)[1:2] <- c('source','target')
ppi_data

In [ ]:
ppi_suid <- createNetworkFromDataFrames(edges=ppi_data,
                            title = 'AP-MS Demo Data-tutorial',
                            collection = 'AP-MS Demo Data')

## Load Expression Data
Now, the network can be annotated with expression data, provided a column in the datset matches a column in the network's Node Table.

In [ ]:
EXPRESSION_DATA_URL = 'https://raw.githubusercontent.com/cytoscape/cytoscape-automation/master/for-scripters/Python/data/annotation-data.csv'
exp_data <- read.csv(EXPRESSION_DATA_URL, stringsAsFactors = F)
exp_data

In this example, the column "GeneSymbol" matches the source nodes in our network in a default "name" column.

In [ ]:
loadTableData(exp_data, data.key.column = 'GeneSymbol')

## Save Network to NDEx
Again, we can save this network to NDEx as well, including the network and associated expression data.

In [ ]:
USERNAME <- readline('Enter your NDEx username: ')
PASSWORD <- readline('Enter your NDEx password: ')
ppi_data_uuid <- exportNetworkToNDEx(USERNAME, PASSWORD, isPublic=FALSE, network=ppi_suid)

# PART III
We have demonstrating working with networks and datasets in Cytoscape via scripting. Of course, you can also represent networks at data objects in your scripting environment for bioinformatic analysis.

## Retrieve Network from NDEx
In order to retrieve networks from NDEx and represent them in R, we will need to install the *ndexr* package.

In [ ]:
if(!"ndexr" %in% installed.packages()){
    install.packages("BiocManager")
    BiocManager::install("ndexr")
}
library(ndexr)

You can start with NDEx by first establishing a connection.

In [ ]:
ndexcon <- ndex_connect()

We will use ndexcon throughout the other ndexr calls. For example, a basic search.

In [ ]:
networks <- ndex_find_networks(ndexcon, "Breast Cancer")
print(networks[,c("name","externalId","nodeCount","edgeCount")])

That print statement provides a nifty way to browse the search results. You’ll notice that we got results that hit each of the search terms individually, thus including any pathway with “cancer” in the name. That’s perhaps a bit too broad…

In [ ]:
networks <- ndex_find_networks(ndexcon, "BRCA")
print(networks[,c("name","externalId","nodeCount","edgeCount")])

Ok. We can work with this list. Let’s use the first hit. Note: you are going to get different hits as this database changes over time, so proceed with any hit you like.

In [ ]:
networkId = networks$externalId[1]
network = ndex_get_network(ndexcon, networkId)
print(network)

The *network* is an RCX object. Explore the RCX object to see its contents.

In [ ]:
str(network)
network$metaData
network$nodes
network$edges

**Note:** RCX can be coverted to two flavors of igraph objects using *rcx_toNGraph* and *rcx_toRCXgraph*. See method documentation for more details.